In [1]:
# This Jupyter Notebook loads and runs various simulations and analyses concerning
# the planaria model reported on in the manuscript, "Neural Control of Body-plan 
# Axis in Regenerating Planaria" by Pietak et al. submitted to PLoS Comp Bio.  

# To use this notebook with the Planaria Interface for Modeling Body Organization 
# (PLIMBO) simulator, please first follow the installation instructions for the 
# BETSE simulator to install necessary dependencies:

# https://gitlab.com/betse/betse

# And then download the PLIMBO project:

# https://gitlab.com/betse/plimbo

# 

# The PLIMBO project folder contains all accessory configuration files and other data 
# required to run planaria model simulations. You can find saved data from 
# simulations that you run in this Notebook (e.g. plots and 
# animations) in the PLIMBO/plimbo/data folder on your system.

In [2]:
# Import statements
import numpy as np
import matplotlib.cm as cm
import matplotlib.pyplot as plt
from matplotlib import colors 
from matplotlib import colorbar
from collections import OrderedDict
from scipy.ndimage import rotate
from scipy.misc import imresize
import copy
import pickle
import os
import os.path
import sys, time
import csv
from betse.lib.pickle import pickles
from matplotlib import rcParams

from plimbo.plimbo import PlimboRunner
from plimbo.sim2D import PlanariaGRN2D

from matplotlib.collections import PolyCollection
from betse.science.math import toolbox as tb

[ipykernel_launcher.py] Loading third-party BETSE dependencies...


In [3]:
# Parameters set-up:

# Finalized parameters for the planaria reaction-convection-diffusion model
# with Markov model parameters included (these parameter values can be edited, but don't
# change the order of the parameters listed):

# All parameters are defined in full (with units specified) in the 
# Supporting Information S1 document. 

param_o = OrderedDict({ 
    'Do': 1.0e-11, # Small general diffusion factor

    # beta-Cat parameters
    'r_bc': 1.5e-3, # max growth rate of beta-Cat [nM/s]
    'd_bc': 5.0e-7, # background decay rate of beta-Cat [1/s]
    'K_bc_apc': 0.5, # K1/2 interaction with APC [nM]
    'n_bc_apc': 1.0, # Hill constant interaction with APC
    'd_bc_deg': 3.0e-3, # APC-induced beta-Cat degradation [1/s] 
    'K_bc_camp': 1.0, # K1/2 interaction with cAMP [uM]
    'n_bc_camp': 2.0, # Hill constant interaction with cAMP
    'D_bc': 1.5e-12, # diffusion constant [m^2/s]

    # ERK parameters
    'r_erk': 5.0e-3, # max growth rate of ERK [nM/s]
    'd_erk': 5.0e-3, # decay rate of ERK [1/s]
    'K_erk_bc': 30.0, # K1/2 interaction with beta-Cat [nM]
    'n_erk_bc': 2.0, 

    # APC parameters
    'r_apc': 5.0e-3, # max growth rate of APC [nM/s]
    'd_apc': 5.0e-3,  # decay rate of APC [1/s]
    'K_apc_wnt': 5.0, # K1/2 interaction with Wnt [nM]
    'n_apc_wnt': 2.0, # Hill constant interaction with Wnt

    # Hedgehog parameters:
    'r_hh': 5.0e-3, # max growth rate of HH [nM/s]
    'd_hh': 1.0e-5,   # decay rate of HH [1/s]
    'D_hh': 2.5e-11,  # diffusion constant [m^2/s]
    'u_hh': 1.25e-7,  # Maximum axoplasmic transport rate [m/s]

    # Wnt parameters
    'r_wnt': 5.0e-3, # max growth rate of Wnt [nM/s]
    'd_wnt': 5.0e-6,  # background decay rate of Wnt [1/s]
    'K_wnt_notum': 0.5, # K1/2 interaction with Notum [nM]
    'n_wnt_notum': 2.0, # Hill constant interaction with Notum
    'D_wnt': 0.75e-11, # diffusion constant [m^2/s]
    'd_wnt_deg_notum': 5.0e-3, # Notum-induced Wnt degradation [1/s] 
    'd_wnt_deg_ptc': 3.5e-5, # Ptc-induced Wnt degradation [1/s] 
    'K_wnt_hh': 62.5, # K1/2 interaction with Hh [nM]
    'n_wnt_hh': 2.0, # Hill constant interaction with Hh
    'K_wnt_camp': 0.5, # K1/2 interaction with cAMP [nM]
    'n_wnt_camp': 2.0, # Hill constant interaction with cAMP

    # Notum regulating factor (NRF) parameters
    'r_nrf': 2.5e-3, # max growth rate of NRF [nM/s]
    'd_nrf': 1.0e-5,  # decay rate of NRF [1/s]
    'K_nrf_bc': 100.0, # K1/2 interaction with beta-Cat [nM]
    'n_nrf_bc': 1.0, # Hill constant interaction with beta-Catenin
    'D_nrf': 2.5e-11, # diffusion constant [m^2/s]
    'u_nrf': -1.0e-7, # Maximum axoplasmic transport rate [m/s]

    # Notum parameters
    'r_notum': 5.0e-3, # max growth rate of Notum [nM/s]
    'd_notum': 5.0e-3,  # decay rate of Notum [1/s]
    'K_notum_nrf': 300.0, # K1/2 interaction with NRF [nM]
    'n_notum_nrf': 3.0,  # Hill constant interaction with NRF
    'D_notum': 2.5e-11, # diffusion constant [m^2/s]

    #cAMP parameters:
    'r_camp': 5.0e-3, # max growth rate of cAMP [nM/s]
    'd_camp': 5.0e-3,  # decay rate of cAMP [1/s]

    # Markov model parameters:
    'C1': 0.50, # ERK concentration to modulate head formation
    'K1': 0.05, # Steepness of transition rate with ERK

    'C2': 75.0, # Beta-catenin concentration to modulate tail formation
    'K2': 4.0,  # Steepness of transition rate with beta-Cat
    
    'Beta_B': 5.0e-3, # head/tail tissue decay time constant

    'max_remod': 1.0e-2,  # maximum rate at which tissue remodelling occurs
    'hdac_growth': 1.0e-3,  # growth and decay constant for hdac remodeling molecule
    'D_hdac': 1.0e-11, # diffusion constant for hdac remodeling molecule
    'hdac_to': 72.0*3600,  # time at which hdac stops growing
    'hdac_ts': 12.0*3600 # time period over which hdac stops growing
})

# A slight modification is required between parameter set for 1D and 2D:
param_1D = param_o.copy()
param_1D['hdac_to'] = 96*3600 # 1D model requires slightly longer healing time.
param_1D['d_wnt_deg_ptc'] = 3.20e-5 # 1D model requires slightly lower wnt degradation 
param_2D = param_o.copy() # 2D model maintains all same parameters as defined above

blnkRNAi = OrderedDict() # empty dictionary for omitting RNAi test sequence in models

In [4]:
# MODEL ACCESS and DATA SAVING
#-----------------------------
# For simplicity, the default 1H worm simulation (4 cuts) will be used here:
model_key = '1HWorm_Default'

# Using the default 1H worm model, data produced by simulations run
# in this Notebook will be saved to (relative to PLIMBO's location on your system):
# e.g. 'PLIMBO/plimbo/data/BasicTesting/1H1/'

# All available model keys can be printed; use one of these to select 
# a different worm model (e.g. model_key = '2HWorm1_1Cut_sym', with head_frags = [0] and
# tail_frags = [1]):

sim = PlimboRunner(head_frags = [0], tail_frags = [4]) 
sim.print_available_models()

# Note that if you want to run, edit the configuration and save a model to a different 
# directory on your system, first copy a whole model sub-folder from the 
# 'PLIMBO/plimbo/data directory' (e.g. copy the entire '1H1Worm' folder) to a new
# location on your system, and then supply a path to the location of the yaml
# file. An an example, if the new yaml is located at: 
# my_config = '/home//Planaria_Project/1H1Worm/1H_ellipse.yaml'
# Then change all calls to the simulator to:
# sim = PlimboRunner(fn_config = my_config, head_frags = [0], tail_frags = [4]) 
# When fn_config is changed from fn_config = None, it will override the worm_model
# key and try to read the model from your specified path. All data will be saved relative
# to where the model yaml config file is on your system.

# Also, head and tail fragments are ordered from the head (0), and the index
# changes depending on how many fragments the worm is cut into. For example, if there 
# are 5 fragments (4 cuts), then the head fragment is indexed 0 and the tail at 4, 
# but if there are 2 fragments (1 cut), then the head fragment is indexed 0 and the tail 1.
# These fragment indices for head and tail must be specified by the user at the time of
# cutting.

Here is a list of worm model keys and a brief description: 
----------------------------------------------------------
1HWorm_Default    Default 1H Worm model cut into 5 fragments.
1HWorm1_4Cuts    1H Worm model 1 cut into 5 fragments.
1HWorm1_2Cuts    1H Worm model 1 cut into 3 fragments.
1HWorm1_1Cut    1H Worm model 1 cut into 2 fragments.
1HWorm2_4Cuts    1H Worm model 2 cut into 5 fragments.
1HWorm2_2Cuts    1H Worm model 2 cut into 3 fragments.
1HWorm3_4Cuts    1H Worm model 3 cut into 5 fragments.
1HWorm3_2Cuts    1H Worm model 3 cut into 3 fragments.
1HWorm5_4Cuts    1H Worm model 5 cut into 5 fragments.
1HWorm5_2Cuts    1H Worm model 5 cut into 3 fragments.
1HWorm5_1Cut    1H Worm model 5 cut into 2 fragments.
2HWorm1_2Cuts_sym    2H Worm model 1 cut into 3 symmetric fragments around midline.
2HWorm1_2Cuts_asym    2H Worm model 1 cut into 3 asymmetric fragments.
2HWorm1_1Cut_sym    2H Worm model 1 cut into 2 symmetric fragments at midline.
2HWorm1_1Cut_asym    2H Worm model 1 

In [ ]:
## The following code blocks can be commented in to run different types of simulations.

## Note that available plot types are:
## 'Triplot', 'Biplot', 'Hexplot' and 'Markovplot' (you may need to adjust 'fsize' for 
## plot type and for different worm models)
## Note available animation types are:
## 'Triplot' and 'Biplot' (you may need to adjust fsize for
## plot type and for different worm models)

In [1]:
# # Run a single body-model simulation in 1D without any RNAi/intervention testing:

# sim = PlimboRunner(worm_model = model_key, fn_config = None,
#                    head_frags = [0], tail_frags = [4]) 
# sim.simRNAi(RNAi_vect = blnkRNAi, RNAi_tags = blnkRNAi, params = param_1D, run_time_init = 3600*168,
#                 run_time_sim = 3600*168, run_time_step = 60, run_time_sample = 25.0,
#                 run_time_reinit = 0.0,  xscale = 1.0, verbose = True, new_mesh = False,
#                 save_dir = 'sim_1D', reset_clims = True, animate = False, plot = True,
#                 plot_type = ['Markovplot','Hexplot'], ani_type = 'Triplot', save_harness = False,
#                 harness_type='1D', fsize = [(12,12), (18,12)])

In [2]:
# # Run a single body-model simulation in 2D without any RNAi/intervention testing:
# sim = PlimboRunner(worm_model = model_key, fn_config = None,
#       head_frags = [0], tail_frags = [4]) 
# sim.simRNAi(RNAi_vect = blnkRNAi, RNAi_tags = blnkRNAi, params = param_2D, run_time_init = 3600*168,
#                 run_time_sim = 3600*168, run_time_step = 60, run_time_sample = 25.0,
#                 run_time_reinit = 0.0,  xscale = 1.0, verbose = True, new_mesh = False,
#                 save_dir = 'sim_2D', reset_clims = False, animate = False, plot = True,
#                 plot_type = ['Markovplot','Hexplot'], ani_type = 'Triplot', save_harness = False,
#                 harness_type='2D', fsize = [(6,8),(6,14)])

In [3]:
# # Run a single body-model simulation with RNAi/intervetions test sequence in 1D:
# sim = PlimboRunner(worm_model = model_key, fn_config = None,
#        head_frags = [0], tail_frags = [4]) 
# sim.simRNAi(params = param_1D, run_time_init = 3600*108,
#                 run_time_sim = 3600*108, run_time_step = 60.0, run_time_sample = 50.0,
#                 run_time_reinit = 0.0,  xscale = 1.0, verbose = True, new_mesh = False,
#                 save_dir = 'SimRNAi_1D', reset_clims = True, animate = False, plot = True,
#                 plot_type = ['Markovplot','Hexplot'], ani_type = 'Triplot', save_harness = False,
#                 harness_type='1D', fsize = [(12,12), (18,12)])

In [4]:
# # Run a single body-model simulation in 2D with RNAi/intervention testing:
# sim = PlimboRunner(worm_model = model_key, fn_config = None,
#                   head_frags = [0], tail_frags = [4]) 
# sim.simRNAi(params = param_2D, run_time_init = 3600*96,
#                 run_time_sim = 3600*96, run_time_step = 60, run_time_sample = 50.0,
#                 run_time_reinit = 0.0,  xscale = 1.0, verbose = True, new_mesh = False,
#                 save_dir = 'simRNAi_2D', reset_clims = False, animate = False, plot = True,
#                 plot_type = ['Markovplot','Hexplot'], ani_type = 'Triplot', save_harness = False,
#                 harness_type='2D', fsize = [(6,8),(6,14)])

In [5]:
# # Run a single body-model simulation with full RNAi tests in 2D with 7 day
# # 'reinitialization' phase:
# sim = PlimboRunner(worm_model = model_key, fn_config = None,
#                    head_frags = [0], tail_frags = [4]) 
# sim.simRNAi(params = param_2D, run_time_init = 3600*96,
#                 run_time_sim = 3600*96, run_time_step = 60.0, run_time_sample = 50.0,
#                 run_time_reinit = 3600.0*168,  xscale = 1.0, verbose = True, new_mesh = False,
#                 save_dir = 'SimRNAi_2D_longreinit', reset_clims = False, animate = True, plot = True,
#                 plot_type = ['Markovplot','Hexplot'], ani_type = 'Triplot', save_harness = False,
#                 harness_type='2D', fsize = [(6,8),(6,14)])

In [6]:
# # Run a sensitivity analysis on the model in 1D:
# sim = PlimboRunner(worm_model = model_key, fn_config = None,
#                   head_frags = [0], tail_frags = [4])
# sim.sensitivity(params=param_1D, 
#                 run_type = 'sim', 
#                 run_time_init=3600*96, 
#                 factor = 0.15,
#                 run_time_sim=3600*96, 
#                 run_time_step=60.0, 
#                 run_time_sample=50.0,
#                  xscale=1.0, 
#                 verbose=True, 
#                 new_mesh=False, 
#                 ani_type = 'Triplot',
#                 save_dir='Sensitivity_1D', 
#                 reset_clims=True, 
#                 animate=False, plot=True,
#                 plot_type=['Markovplot','Hexplot'],
#                 save_harness=False, 
#                 harness_type='1D',
#                 reference = ['Head','Tail','Erk','β-Cat','Notum','Hh'], 
#                 fsize = [(12,12), (18,12)],
#                 paramo_units = None)

In [7]:
# # Run a sensitivity analysis on the model in 2D:
# sim = PlimboRunner(worm_model = model_key, fn_config = None,
#                    head_frags = [0], tail_frags = [4])
# sim.sensitivity(params=param_2D, 
#                 run_type = 'sim', 
#                 run_time_init=3600*96, 
#                 factor = 0.15,
#                 run_time_sim=3600*96, 
#                 run_time_step=60.0, 
#                 run_time_sample=50.0,
#                  xscale=1.0, 
#                 verbose=True, 
#                 new_mesh=False, 
#                 ani_type = 'Triplot',
#                 save_dir='Sensitivity_2D', 
#                 reset_clims=True, 
#                 animate=False, 
#                 plot=True,
#                 plot_type=['Markovplot','Hexplot'], 
#                 save_harness=False, 
#                 harness_type='2D',
#                reference = ['Head','Tail','Erk','β-Cat','Notum','Hh'], 
#                 fsize = [(6,8),(6,14)], 
#                 axisoff = True, 
#                 )

In [12]:
# # Run a local parameter search for model improvements in 1D:
# sim = PlimboRunner(worm_model = model_key, fn_config = None,
#                   head_frags = [0], tail_frags = [4]) 
# sim.searchRNAi(params=param_1D, free_params = None, run_time_init = 3600.0*96,
#                 run_time_sim = 3600.0*96, run_time_step = 90.0, run_time_sample = 50.0, search_style = 'log',
#                    factor = 0.66, levels = 1, run_time_reinit = 0.0, xscale = 1.0, verbose = True, new_mesh = False,
#                 save_dir = 'SearchRNAi_1D', reset_clims = True, animate = False, plot = True, ani_type = 'Triplot',
#                 plot_type = ['Markovplot','Hexplot'], save_harness = False, harness_type='1D',
#                   fsize = [(12,12), (18,12)])

In [13]:
# # Run a local parameter search for model improvements in 2D:
# sim = PlimboRunner(worm_model = model_key, fn_config = None,
#                     head_frags = [0], tail_frags = [4]) 
# sim.searchRNAi(params=param_2D, 
#                free_params = None, 
#                xscale = 1.0, 
#                harness_type='2D',
#                save_dir = 'SearchRNAi_2D', 
#                run_time_init = 3600.0*96, 
#                run_time_reinit = 0.0, 
#                run_time_sim = 3600.0*96, 
#                run_time_step = 90.0, 
#                run_time_sample = 50.0, 
#                search_style = 'log',
#                factor = 0.75, 
#                levels = 2, 
#                verbose = True, 
#                new_mesh = False,
#                reset_clims = True, 
#                animate = False, 
#                plot = True, 
#                ani_type = 'Triplot',
#                plot_type = ['Markovplot','Hexplot'],
#                save_harness = False, 
#                axisoff = False,
#                fsize = [(6,8),(6,14)],
#                up_only = False)